# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_1 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project_1')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [8]:
query = """CREATE TABLE IF NOT EXISTS music_app_history (
        sessionid INT,
        iteminsession INT,
        userid INT,
        artist TEXT,
        songtitle TEXT,
        firstname TEXT,
        lastname TEXT,
        length DOUBLE,
        gender TEXT,
        level TEXT,
        location TEXT,
        PRIMARY KEY ((sessionid, iteminsession), userid)
    ) """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
query = """CREATE TABLE IF NOT EXISTS songsbyusersession (
            userid INT,
            sessionid INT,
            iteminsession INT,
            artist TEXT,
            songtitle TEXT,
            firstname TEXT,
            lastname TEXT,
            length DOUBLE,
            gender TEXT,
            level TEXT,
            location TEXT,
            PRIMARY KEY (userid, sessionid, iteminsession)
        )"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
query = """CREATE TABLE IF NOT EXISTS usersbysong (
            songtitle TEXT,
            userid INT,
            firstname TEXT,
            lastname TEXT,
            PRIMARY KEY (songtitle, userid)
        )"""
try:
    session.execute(query)
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (sessionid, iteminsession, userid, artist, songtitle, firstname, lastname, length, gender, level, location) "
        query += "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (
        int(line[8]), int(line[3]), int(line[10]),
        line[0], line[9], line[1], line[4],
        float(line[5]), line[2], line[6],
        line[7]
        )

        session.execute(query, values)

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songsbyusersession (sessionid, iteminsession, userid, artist, songtitle, firstname, lastname, length, gender, level, location) "
        query += "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (
        int(line[10]), int(line[8]), int(line[3]),
        line[0], line[9], line[1], line[4],
        float(line[5]), line[2], line[6],
        line[7]
        )

        session.execute(query, values)

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO usersbysong (songtitle, userid, firstname, lastname) "
        query += "VALUES (%s, %s, %s, %s)"
        values = (
        line[9], int(line[10]), line[1], line[4]
        )

        session.execute(query, values)

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
# TO-DO: Query 1: Give me the artist, song title, and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "SELECT artist, songtitle, length FROM music_app_history WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.songtitle, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [15]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "SELECT artist, songtitle, firstname, lastname FROM songsbyusersession WHERE userid = 10 AND sessionid > 1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.songtitle, row.firstname, row.lastname)

                    

T. Rex Ride A White Swan Lily Koch
Sean Lennon Into The Sun Lily Koch
BeyoncÃÂ© Halo Lily Koch
Billy Currington Good Directions Lily Koch
Kings Of Leon The Runner Lily Koch
Aphex Twin Heliospan Lily Koch
Lara & Reyes El Castillo Lily Koch
Sugar Ray Someday (Remastered LP Version) Lily Koch
Craig Chaquico Kyle's World Rylan George
Juanes No Siento Penas Rylan George
Kiuas The New Chapter Rylan George
Carla Thomas A Woman's Love   (LP Version) Rylan George
Soul For Real Candy Rain Rylan George
Dwight Yoakam You're The One Layla Griffin
Mickie Krause Orange TrÃÂ¤gt Nur Die MÃÂ¼llabfuhr (Go West) Layla Griffin
Sonny & Cher What Now My Love (LP Version) Layla Griffin
Robert Rich & Alio Die Sirena Layla Griffin
Bebo Norman Perhaps She'll Wait Layla Griffin
Scissor Sisters I Don't Feel Like Dancin' Layla Griffin
The Romantics Talking In Your Sleep Jayden Graves
Sia Little Black Sandals Jayden Graves
Pat Martino El Hombre Jayden Graves
Dio When A Woman Cries Jayden Graves
Eddie Vedder Guara

In [16]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT firstname, lastname FROM usersbysong WHERE songtitle = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, row.lastname)
                   

Jacqueline Lynch
Tegan Levine
Sara Johnson


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [17]:
# TO-DO: Query 1: Give me the artist, song title, and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "SELECT artist, songtitle, length FROM music_app_history WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.songtitle, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [18]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "SELECT artist, songtitle, firstname, lastname FROM songsbyusersession WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.songtitle, row.firstname, row.lastname)

                    

In [19]:
#### Apparently there is no item which matches the criteria in the where clause in the above query.

In [20]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT firstname, lastname FROM usersbysong WHERE songtitle = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, row.lastname)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
query = "DROP table music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP table songsbyusersession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

query = "DROP table usersbysong"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()